In [1]:
!mkdir -p input
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z01 -O input/news.csv.z01
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z02 -O input/news.csv.z02
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z03 -O input/news.csv.z03
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z04 -O input/news.csv.z04
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z05 -O input/news.csv.z05
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z06 -O input/news.csv.z06
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z07 -O input/news.csv.z07
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z08 -O input/news.csv.z08
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.z09 -O input/news.csv.z09
!wget -q https://github.com/several27/FakeNewsCorpus/releases/download/v1.0/news.csv.zip -O input/news.csv.zip

In [2]:
!cat input/news.csv.z* > input/all.zip

In [3]:
!unzip input/all.zip -d input/

Archive:  input/all.zip
error: End-of-centdir-64 signature not where expected (prepended bytes?)
  (attempting to process anyway)
warning [input/all.zip]:  zipfile claims to be last disk of a multi-part archive;
  attempting to process anyway, assuming all parts have been concatenated
  together in order.  Expect "errors" and warnings...true multi-part support
  doesn't exist yet (coming soon).
warning [input/all.zip]:  9663676416 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  9663676420
  (attempting to re-compensate)
  inflating: input/news_cleaned_2018_02_13.csv  


In [6]:
!rm input/news.csv.z* input/all.zip

rm: cannot remove 'input/all.zip': No such file or directory


In [7]:
import tensorflow as tf
import pandas as pd
import itertools
import random
import sys
import csv

In [8]:
numrows = 8528956

In [9]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [10]:
labels = []

def get_label(label):
    if label == "":
        label = "unknown"
    if label == "2018-02-10 13:43:39.521661":
        label = "unknown"
    if label not in labels:
        labels.append(label)
    return labels.index(label)

In [11]:
def serialize_example(text, label):
  feature = {
      "text": _bytes_feature(text.encode("utf-8")),
      "label": _int64_feature(get_label(label)),
  }

  example = tf.train.Example(features=tf.train.Features(feature=feature))
  return example.SerializeToString()

In [42]:
project_id = "alaska2-280023"
bucket_name = "dbunk"

In [41]:
from google.colab import auth
auth.authenticate_user()

In [43]:
!gcloud config set project {project_id}

Updated property [core/project].


In [16]:
!rm *.tfrecord

In [17]:
numwriters = numrows // 65536
writers = [tf.io.TFRecordWriter(f"{i}.tfrecord") for i in range(numwriters)]
nums = [0 for i in range(numwriters)]

In [ ]:
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [21]:
with open("input/news_cleaned_2018_02_13.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    reader = (row for row in reader if row["content"])

    for idx, row in enumerate(reader):
        if idx % (numwriters * 2048) == 0:
            print("writing " + str(idx))

        if idx >= 65536 * numwriters:
            print("broke at " + str(idx))
            break

        widx = idx % numwriters

        if row["title"]:
            text = row["title"] + "\n\n" + row["content"]
        else:
            text = row["content"]
        
        example = serialize_example(text[:1000], row["type"])
        writers[widx].write(example)
        nums[widx] += 1

writing 0
writing 266240
writing 532480
writing 798720
writing 1064960
writing 1331200
writing 1597440
writing 1863680
writing 2129920
writing 2396160
writing 2662400
writing 2928640
writing 3194880
writing 3461120
writing 3727360
writing 3993600
writing 4259840
writing 4526080
writing 4792320
writing 5058560
writing 5324800
writing 5591040
writing 5857280
writing 6123520
writing 6389760
writing 6656000
writing 6922240
writing 7188480
writing 7454720
writing 7720960
writing 7987200
writing 8253440
writing 8519680
broke at 8519680


In [22]:
nums

[65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,
 65536,


In [44]:
!gsutil -m cp *.tfrecord gs://dbunk

Copying file://100.tfrecord [Content-Type=application/octet-stream]...
Copying file://0.tfrecord [Content-Type=application/octet-stream]...
Copying file://107.tfrecord [Content-Type=application/octet-stream]...
Copying file://108.tfrecord [Content-Type=application/octet-stream]...
Copying file://104.tfrecord [Content-Type=application/octet-stream]...
Copying file://106.tfrecord [Content-Type=application/octet-stream]...
Copying file://105.tfrecord [Content-Type=application/octet-stream]...
Copying file://101.tfrecord [Content-Type=application/octet-stream]...
Copying file://103.tfrecord [Content-Type=application/octet-stream]...
Copying file://102.tfrecord [Content-Type=application/octet-stream]...
Copying file://109.tfrecord [Content-Type=application/octet-stream]...
Copying file://10.tfrecord [Content-Type=application/octet-stream]...
Copying file://110.tfrecord [Content-Type=application/octet-stream]...
Copying file://111.tfrecord [Content-Type=application/octet-stream]...
Copying f

In [36]:
for i in writers:
    i.close()

In [ ]:
labels

['rumor',
 'hate',
 'unreliable',
 'conspiracy',
 'clickbait',
 'satire',
 'fake',
 'reliable',
 'bias',
 'political',
 'junksci',
 'unknown']